In [6]:
import pandas as pd
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import os
import random

In [113]:
plt.figure(figsize=(20,10))
test_folder='../../data/raw/example_image/'
file = os.listdir(test_folder)
image_path= os.path.join(test_folder,file[2])
img=plt.imread(image_path)
print(img)
#for i in file[1:5]:
#    image_path= os.path.join(test_folder,i)
#    img=plt.imread(image_path)
#    ax = plt.subplots()
#    plt.imshow(img);

[[[137 125 109]
  [138 124 111]
  [140 126 115]
  ...
  [138 113  93]
  [136 111  91]
  [136 110  93]]

 [[136 124 108]
  [138 125 109]
  [139 125 112]
  ...
  [134 111  93]
  [136 110  93]
  [136 110  93]]

 [[138 125 109]
  [138 125 109]
  [136 123 107]
  ...
  [136 110  93]
  [136 110  93]
  [136 110  93]]

 ...

 [[138 111  90]
  [137 110  89]
  [138 111  92]
  ...
  [141 118 102]
  [140 117 101]
  [141 118 104]]

 [[137 112  90]
  [138 111  92]
  [138 111  94]
  ...
  [142 119 103]
  [140 117 101]
  [140 117 101]]

 [[136 114  93]
  [137 111  94]
  [139 112  95]
  ...
  [142 119 103]
  [141 118 102]
  [140 117  99]]]


<Figure size 1440x720 with 0 Axes>